In [1]:
import pandas as pd
import os
import numpy as np
from dataclasses import dataclass
import random
import time



np.random.seed(14)

/tmp/ipykernel_1607/1459829418.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
@dataclass
class Particle:
    x: float
    y: float
    theta: float
    weight: float 
        


@dataclass
class LandMark:
    x: float
    y: float
    index: int

In [3]:
# location of car in (x,y,orientation)
gt_data = pd.read_csv('data/gt_data.txt', names=['X','Y','Orientation'], sep=' ')
#list of landmarks in the map [x,y,# landmark]
map_data = pd.read_csv('data/map_data.txt', names=['X','Y','# landmark'])
# velocity and theta of the car
control_data = pd.read_csv('data/control_data.txt', names=['velocity','Yaw rate'], sep=' ')

#observation = pd.read_csv('data/observation/observations_000001.txt', names = ['X cord','Y cord'], sep=' ')


result = [(x,y, landmark) for x,y,landmark in zip(map_data['X'],map_data['Y'], map_data['# landmark'])]
landarkList=[]
for res in result:
    l = LandMark(res[0],res[1],res[2])
    landarkList.append(l)

    

a = os.listdir("data/SensorDataFiles")
a.sort()
#list of  items observations from car point of view
observation=[]
for i in range(len(a)):
    fileName = 'data/SensorDataFiles/'+a[i]
    observationTmp = pd.read_csv(fileName, names = ['X cord','Y cord'], sep=' ')
    observation.append(observationTmp)

print(observation[1])
relevent_x = observation[1]['X cord'].values.tolist()
print(relevent_x)
print(control_data)



     X cord    Y cord
0    3.6700   4.58240
1    8.2123  -9.63970
2  -20.0050   4.10470
3   -5.5529 -22.42600
4    6.0674 -25.28000
5   32.0180   3.14540
6  -24.2870 -30.68500
7   19.1080 -38.63300
8  -42.4870 -13.04300
9    8.4421 -46.62000
10 -48.7180   0.14176
[3.67, 8.2123, -20.005, -5.5529, 6.0674, 32.018, -24.287, 19.108, -42.487, 8.4421, -48.718]
      velocity  Yaw rate
0      3.96110  3.093700
1      4.03780 -0.008101
2      4.11730  0.005551
3      4.20660  0.011087
4      4.30070 -0.001022
...        ...       ...
2439   7.70090 -0.079940
2440   7.68990 -0.085936
2441   7.67620 -0.071506
2442   7.65070 -0.065456
2443   0.45249 -8.912000

[2444 rows x 2 columns]


In [4]:
def calculateDistance(landmark1, landmark2):
    
    x_1 = landmark1.x
    y_1 = landmark1.y
    sum = 0
    #x_2 = landmark2['X cord'].values.tolist()
    #y_2 = landmark2['Y cord'].values.tolist()
    for i in range(0,len(landmark2)):
        sum+= (x_1 - landmark2[i].x)**2 + (y_1 - landmark2[i].y)**2
    sum = np.sqrt(sum)
    return sum
    #return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

    #a =  np.sqrt((landmark1.x - landmark2.x)**2 + (landmark1.y - landmark2.y)**2)
    #return a
    
 
    
def findClosestLandmark(map_landmarks, singleObs):
    
   #todo: write code:
    min_distance = float('inf')
    closest_landmark = None

    for landmark in map_landmarks:
        distance = calculateDistance(landmark, singleObs)
        if distance < min_distance:
            min_distance = distance
            closest_landmark = landmark 
       
    return closest_landmark
    
    
    
        
def getError(gt_data, bestParticle):
    error1 = np.abs(gt_data[0] - bestParticle.x)
    error2 = np.abs(gt_data[1] - bestParticle.y)
    error3 = np.abs(gt_data[2] - bestParticle.theta)
    if(error3>2*np.pi):
        error3 = 2*np.pi - error3
    return (error1, error2, error3)

def findObservationProbability(closest_landmark,map_coordinates, sigmaX, sigmaY):
    
    #print("particle",closest_landmark)
    #print("map", map_coordinates)
    mew_x = closest_landmark.x
    mew_y = closest_landmark.y
    x = map_coordinates.x;
    y = map_coordinates.y;
    frac =  (1/ np.sqrt(2 * np.pi * sigmaX * sigmaY ))
    weight1 = np.sqrt((x-mew_x)**2/((sigmaX)**2)  +(y-mew_y)**2/(sigmaY**2))   
    ans = np.exp(-0.5*weight1)* frac
    #print("mew_x ", mew_x,"mew_y ",mew_y,"x ",x,"y",y,"frac ", frac,"weight1 ", weight1,"ans ", ans)
    #print("ex ",np.exp(-0.5*weight1))
    return ans



def mapObservationToMapCoordinates(observation, gt_data):
    x = observation.x
    y = observation.y
    gt_x = gt_data['X']
    gt_y = gt_data['Y']
    gt_theta = gt_data['Orientation']
    xt = gt_x
    yt = gt_y
    theta = gt_theta

    MapX = x * np.cos(theta) - y * np.sin(theta) + xt
    MapY = x * np.sin(theta) + y * np.cos(theta) + yt

    return MapX, MapY

def mapObservationsToMapCordinatesList(observations, gt_data):
    
    convertedObservations=[]
    i=0

    x = observations['X cord'].values.tolist()
    y = observations['Y cord'].values.tolist()

    for obs in range(0,len(x)):
        singleObs = LandMark(x[obs],y[obs],1)
        mapX, mapY = mapObservationToMapCoordinates(singleObs, gt_data)
        tmpLandmark = LandMark(x=mapX, y=mapY, index=i)
        i+=1
        convertedObservations.append(tmpLandmark)
    return convertedObservations



In [5]:
class ParticleFilter:
    particles = []
    def __init__(self, intialX, initialY, std, numOfParticles):
        self.number_of_particles = numOfParticles
        for i in range(self.number_of_particles):
           # tmpParticle = Particle(intialX,initialY,0,1)
            x = random.gauss(intialX, std)
            y = random.gauss(initialY, std)
            theta = random.uniform(0, 2*np.pi)
            tmpParticle = Particle(x,y , theta,1)
            self.particles.append(tmpParticle)
            
            
            
    def moveParticles(self, velocity, yaw_rate, delta_t=0.1):
        
        for i in range(self.number_of_particles):
            if(yaw_rate!=0):
                theta = self.particles[i].theta
                newTheta = theta + delta_t*yaw_rate;
                newX =  self.particles[i].x +(velocity/yaw_rate)*(np.sin(newTheta)-np.sin(theta));
                newY =  self.particles[i].y +(velocity/yaw_rate)*(np.cos(theta)-np.cos(newTheta));
                
                #todo Add noise!!
                self.particles[i].x = newX +random.gauss(0, 0.3)
                self.particles[i].y = newY +random.gauss(0, 0.3)
                self.particles[i].theta = newTheta +random.gauss(0, 0.01)
            else:
                print("ZERO!!!")
            

        
        
    def UpdateWeight(self, observations):
        
           #todo: Write code:
        for particle in range(0,self.number_of_particles):
            total_weight = 0.0
            #for obs in observations:
            #closest landmark to given observation in time t. each observation is already been maped to actual location on global map relative to car position
            closest_landmark = findClosestLandmark(landarkList, observations)
            map_coordinates = LandMark(closest_landmark.x, closest_landmark.y, closest_landmark.index)
            sigmaX = 0.3  # You may need to adjust these values
            sigmaY = 0.3  # based on your application
            observation_prob = findObservationProbability(self.particles[particle], map_coordinates, sigmaX, sigmaY)
            total_weight += observation_prob
            self.particles[particle].weight = total_weight
    
            
    
    def getBestParticle(self):
        best_particle =  max(self.particles, key= lambda particle: particle.weight)
        return best_particle    
    
    def getBestParticleOut(self):
        x=0
        y=0
        theta=0
        for i in range(self.number_of_particles):
            x+= self.particles[i].x
            y+= self.particles[i].y
            theta+= self.particles[i].theta
        x=x/self.number_of_particles
        y=y/self.number_of_particles
        theta=theta/self.number_of_particles
        best_particle =  Particle(x,y,theta, weight=1)
        return best_particle        
    
    def PrintWeights(self):
        for i in range(self.number_of_particles):
            print("Weight:",self.particles[i].weight, self.particles[i].x,self.particles[i].y)
    
    
    def Resample(self):
        
    #todo: writecode
        weights = [self.particles[particle].weight for particle in range(0,self.number_of_particles)]
        #print("we", weights)
        max_weight = max(weights)
        #print("ma ", max_weight )
        index = random.randint(0, self.number_of_particles - 1)
        offset = 0.0
        new_particles = []

        for i in range(0,self.number_of_particles):
            offset += random.uniform(0, 2 * max_weight)
            while offset > weights[index]:
                offset -= weights[index]
                index = (index + 1) % self.number_of_particles
            new_particles.append(self.particles[index])
        #print("Ds",new_particles)

        self.particles = new_particles
    
       
        
        

In [7]:
sigmaY = 0.3

magicNumberOfParticles = 200


start = time.time()

def main():
    particleFilter = ParticleFilter(0 ,0 ,0.3, numOfParticles=magicNumberOfParticles)
    #particleFilter = ParticleFilter(6.2785 ,1.9598 ,0.3, numOfParticles=magicNumberOfParticles)


    for i in range(0,130):
        #prediction
        if(i!=0):
            velocity = control_data.iloc[i-1][0]
            yaw_rate = control_data.iloc[i-1][1]
            particleFilter.moveParticles(velocity, yaw_rate)
        a = observation[i].copy()
        observations_in_index = mapObservationsToMapCordinatesList(a, gt_data.iloc[i])
        #print("a ",a)
        #print("\n")
        #print("after ",observations_in_index)
        particleFilter.UpdateWeight(observations_in_index)
        particleFilter.Resample()
        bestP = particleFilter.getBestParticle()
        error = getError(gt_data.iloc[i], bestP)
        print(i,error)
    end = time.time()
    print(end - start)

main()


/tmp/ipykernel_1607/741004142.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  error1 = np.abs(gt_data[0] - bestParticle.x)
/tmp/ipykernel_1607/741004142.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  error2 = np.abs(gt_data[1] - bestParticle.y)
/tmp/ipykernel_1607/741004142.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  error3 = np.abs(gt_data[2] - bestParticle.theta)
/tmp/ipykernel_1607/3748333318.py:16: FutureWarn

0 (4.021258298599335, 6.274090400294391, 5.225922419984461)
1 (4.393820315378573, 5.27903801580263, -0.008866784810285466)
2 (5.461292221660554, 5.750913695099453, 5.3308329616897945)
3 (1.824184679011445, 4.933941202612749, -0.13135494173850137)
4 (10.107247645968162, 4.961881486034701, -40.79274805914957)
5 (95.10282912472077, 35.03867586993407, -40.9457328927044)
6 (182.8420001613635, 62.11753776610163, -40.76445186566562)
7 (271.4240490790231, 81.65016331018784, -40.65736408557823)
8 (367.1136391851465, 79.55707771532927, -40.35042754503382)
9 (462.31468453714854, 71.5976414398038, -40.6736243249167)
10 (563.3541416332827, 89.0607129730708, -40.8908365888161)
11 (658.9967338028362, 99.38126045080455, -40.449317767908525)
12 (758.9584222198781, 89.72126331785462, -40.26370757102815)
13 (853.1297592848225, 39.1897888618881, -40.01476685368929)
14 (941.6357428624214, 2.054413306310135, -39.97932726788755)
15 (1021.9776089632853, 65.47992931895222, -39.80104208662999)
16 (1082.43484496

In [118]:
import cv2

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# WIDTH=1600
# HEIGHT=1200

# def mouseCallback(event, x, y, flags,null):
#     global i
    

# WINDOW_NAME="Particle Filter"
# img = np.zeros((HEIGHT,WIDTH,3), np.uint8)
# cv2.namedWindow(WINDOW_NAME)
# DELAY_MSEC=50
# #cv2.setMouseCallback(WINDOW_NAME,mouseCallback)

# i=1


    

# particleFilter = ParticleFilter(6.2785 ,1.9598 ,0.3, numOfParticles=50)

# while(1):
 
#     cv2.imshow(WINDOW_NAME,img)
#     img = np.zeros((HEIGHT,WIDTH,3), np.uint8)
    
#     if(i>2000):
#         break
    
#     for landmark in landarkList:
#         cv2.circle(img,tuple((int(landmark.x)*2+400, int(landmark.y)*2+300)),3,(255,0,0),-1)
    
#     #draw_particles:
#     for particle in particleFilter.particles:
#         cv2.circle(img,tuple((int(particle.x)*2+400,int(particle.y)*2+300)),1,(255,255,255),-1)
#     cv2.circle(img,tuple((int(gt_data.iloc[i].X)*2+400,int(gt_data.iloc[i].Y)*2+300)),1,(0,255,0),-1)
    
#     if cv2.waitKey(DELAY_MSEC) & 0xFF == 27:
#         break
#     if cv2.waitKey(33) == ord('a'):
#         velocity = control_data.iloc[i-1][0]
#         yaw_rate = control_data.iloc[i-1][1]
#         particleFilter.moveParticles(velocity, yaw_rate)
#         a = observation[i].copy()
#         particleFilter.UpdateWeight(a)
#         particleFilter.PrintWeights()
#         particleFilter.Resample()
        
#         i+=1

# cv2.destroyAllWindows()
